### Libraries and dependencies

In [ ]:
# Import libraries
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

### Import and convert dataset

In [ ]:
# Import dataset
NASA_dataset = pd.read_csv("/content/drive/MyDrive/IR-SEED/B0005.csv")

# Shuffle dataset
NASA_dataset = NASA_dataset.sample(frac=1, random_state=64)

# Drop unecessary columns
NASA_dataset = NASA_dataset.drop(["Current_charge", "Voltage_charge", "type", "ambient_temperature", "time", "Battery"], axis=1)

# Split to training and test sets
training_set = NASA_dataset[:int(len(NASA_dataset) * 0.8)]
testing_set = NASA_dataset[int(len(NASA_dataset) * 0.8):]

# Extract features
train_in = training_set.drop("Capacity", axis=1)
test_in = testing_set.drop("Capacity", axis=1)

# Extract labels
train_out = training_set["Capacity"]
test_out = testing_set["Capacity"]

# Convert features to tensor
train_in = tf.constant(train_in)
train_out = tf.constant(train_out)

# Convert labels to tensor
test_in = tf.constant(test_in)
test_out = tf.constant(test_out)

In [ ]:
# Build model
soh_estimator = tf.keras.Sequential() # Instantiate model

# Add layers
soh_estimator.add(tf.keras.layers.Dense(units=100, input_shape=(5,))) # Hidden layer
soh_estimator.add(tf.keras.layers.Dense(1)) # Output

# Compile model
soh_estimator.compile(loss="mae",
                      optimizer="adam",
                      metrics="mae")

In [ ]:
# Fit the model
loss_hist = soh_estimator.fit(input_train, output_train, epochs=10)

In [ ]:
input_test[0].numpy()

In [ ]:
tf.keras.utils.plot_model(model=soh_estimator, show_shapes=True)

In [ ]:
pd.DataFrame(loss_hist.history).plot(figsize=(10,7))
plt.ylabel("Loss")
plt.xlabel("Epochs")

In [ ]:
soh_estimator.predict(input_test)

In [ ]:
output_test

In [ ]:
soh_estimator.evaluate(input_test, output_test)